In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("all_kindle_review .csv")
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [3]:
df = df[['reviewText','rating']]


In [4]:
df.shape

(12000, 2)

In [5]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [6]:
pd.unique(df['rating'])

array([3, 5, 4, 2, 1], dtype=int64)

In [7]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [8]:
##preprocessing and cleaning 
df['rating']= df['rating'].apply(lambda x :0 if x<3 else 1)

In [9]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [10]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [11]:
## 1.Lower All the cases 
df['reviewText'] = df['reviewText'].str.lower()

In [12]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [14]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup



In [15]:
df = df 
## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))



In [16]:
## Remove the stopswords
stop_words = stopwords.words('english')
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stop_words]))

In [17]:
df = df

## Remove url 
# This pattern just removes anything that starts with http/https/ftp/ssh and continues until a space
url_pattern = r'\b(?:http|https|ftp|ssh)://\S+'

df['reviewText'] = df['reviewText'].astype(str).apply(lambda x: re.sub(url_pattern, '', x))


In [18]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [19]:
df = df
from bs4 import BeautifulSoup

df['reviewText'] = df['reviewText'].apply(lambda x: BeautifulSoup(str(x), 'html.parser').get_text())


In [20]:

## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

In [21]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [22]:
def lemmatize(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [23]:
df['reviewText'] = df['reviewText'].apply(lambda x : lemmatize(x))

In [24]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['reviewText'],df['rating'],
                                              test_size=0.20)

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()

In [26]:
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [28]:
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

In [29]:
from sklearn.naive_bayes import GaussianNB

In [30]:
NGb= GaussianNB()

In [31]:
model_bow = NGb.fit(X_train_bow,y_train)

In [32]:
model_tfidf = NGb.fit(X_train_tfidf,y_train)

In [33]:
y_predbow = NGb.predict(X_test_bow)

In [34]:
y_predtfidf = NGb.predict(X_test_tfidf)

In [35]:
from sklearn.metrics import accuracy_score, classification_report

print("classification report for bow \n",classification_report(y_predbow,y_test))
print(accuracy_score(y_predbow,y_test))

classification report for bow 
               precision    recall  f1-score   support

           0       0.60      0.44      0.51      1073
           1       0.63      0.76      0.69      1327

    accuracy                           0.62      2400
   macro avg       0.62      0.60      0.60      2400
weighted avg       0.62      0.62      0.61      2400

0.6208333333333333


In [36]:
print("classification report for tfidf \n",classification_report(y_predtfidf,y_test))
print(accuracy_score(y_test,y_predtfidf))

classification report for tfidf 
               precision    recall  f1-score   support

           0       0.65      0.41      0.51      1238
           1       0.55      0.76      0.64      1162

    accuracy                           0.58      2400
   macro avg       0.60      0.59      0.57      2400
weighted avg       0.60      0.58      0.57      2400

0.5829166666666666


In [37]:
##converting text to number using word2vec
import gensim 

In [38]:
from nltk.tokenize import word_tokenize
import nltk 
nltk.download('punkt')

df['tokens'] = df['reviewText'].apply(lambda x : word_tokenize(x))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Adwait\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [39]:
from gensim.models import Word2Vec

In [40]:
sentences = df['tokens'].tolist()

model = Word2Vec(sentences,vector_size=100,window=5)

In [41]:
model


In [42]:
import numpy as np

def get_avg_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

# This applies the function correctly (calls the function, doesn't store the function itself)
df['avg_vector'] = df['tokens'].apply(lambda x: get_avg_vector(x, model))


In [43]:
X = pd.DataFrame(df['avg_vector'].tolist())
y = df['rating']  # Assuming your labels are in this column


In [44]:
from sklearn.model_selection import train_test_split


In [45]:
X_train , X_test , y_train ,y_test = train_test_split(X,y,test_size=0.20 )

In [46]:
from sklearn.naive_bayes import GaussianNB

ngb = GaussianNB()

In [47]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
7477,0.031870,0.153595,0.034868,0.091607,0.183315,-0.505403,0.102987,0.566778,-0.392206,-0.125976,...,0.495605,0.183430,0.059332,0.039866,0.363080,0.296209,0.007709,-0.430293,0.206902,-0.085627
62,0.063659,0.207097,0.037425,0.108445,0.253362,-0.630925,0.097063,0.760227,-0.324619,-0.127446,...,0.592008,0.255781,0.046298,0.013324,0.309858,0.297073,0.018175,-0.619000,0.197595,-0.116424
9330,0.119752,0.138377,0.044918,0.182345,0.296613,-0.812950,0.450605,0.909630,-0.489970,0.019840,...,0.612919,0.200950,0.248688,0.108993,0.366144,0.069498,0.206948,-0.393591,0.159524,-0.458092
1341,-0.066389,0.239307,0.055258,0.047815,0.150196,-0.649557,0.374650,0.829708,-0.341496,-0.142520,...,0.450305,0.181762,0.174110,-0.079512,0.350370,0.154554,0.169202,-0.358341,0.140123,-0.061426
11216,-0.023177,0.134699,0.007769,0.255799,0.030593,-0.728270,0.424005,0.668154,-0.360780,-0.087812,...,0.511580,0.107538,0.060377,-0.087743,0.345276,0.189754,0.069502,-0.487765,0.132235,-0.234237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10656,0.084726,0.193933,0.034870,0.071381,0.161031,-0.721318,0.115526,0.651725,-0.371864,-0.104425,...,0.668609,0.093625,0.143771,0.073831,0.345694,0.304781,-0.005289,-0.409891,0.208325,-0.250919
4942,-0.125179,0.203765,0.074738,0.095908,0.057890,-0.540790,0.312738,0.604439,-0.420494,-0.162307,...,0.361349,0.016151,0.092623,0.015244,0.429464,0.240071,0.072835,-0.311303,0.075358,-0.058908
7691,0.004447,0.138756,0.087096,0.101884,0.133817,-0.617472,0.386872,0.693681,-0.357562,-0.129898,...,0.463649,0.151054,0.143795,-0.031055,0.416402,0.179712,0.160747,-0.287655,0.225658,-0.203899
6573,-0.081307,0.261498,-0.023893,0.005012,0.116216,-0.506964,0.120387,0.578305,-0.354936,-0.121205,...,0.360472,0.088609,0.131216,0.069577,0.355632,0.249076,0.068830,-0.273994,0.176038,-0.067152


In [48]:
ngb.fit(X_train,y_train)

,priors,None
,var_smoothing,1e-09


In [49]:
y_pred = ngb.predict(X_test)

In [50]:
from sklearn.metrics import accuracy_score,classification_report

In [51]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.57      0.77      0.65       819
           1       0.85      0.70      0.77      1581

    accuracy                           0.72      2400
   macro avg       0.71      0.73      0.71      2400
weighted avg       0.76      0.72      0.73      2400



In [52]:
print(accuracy_score(y_test,y_pred))

0.7208333333333333
